In [1]:
import ee
ee.Authenticate(auth_mode='notebook')
ee.Initialize(project='global-booster-421311')


In [2]:
catchments = ee.FeatureCollection("projects/global-booster-421311/assets/LakeCatchments_NA")
landcover_codes = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 170, 180]
export_fields = ['Lake_ID', 'year', 'week'] + [f'burned_pix_lc_{lc}' for lc in landcover_codes]

# Specify your years here
years = list(range(2002, 2020))  # 2002 to 2020

def features_for_week(image, year, week):
    week = ee.Number(week)
    start_day = week.subtract(1).multiply(7).add(1)
    end_day = week.multiply(7).min(366)
    burned = image.select('BurnDate').gte(start_day).And(image.select('BurnDate').lte(end_day))
    landcover = image.select('LandCover')
    bands = [burned.And(landcover.eq(lc)).rename(f'burned_pix_lc_{lc}') for lc in landcover_codes]
    stack = ee.Image.cat(bands)
    stats = stack.reduceRegions(
        collection=catchments,
        reducer=ee.Reducer.sum().forEachBand(stack),
        scale=250
    ).map(lambda f: f.set({'year': year, 'week': week}))
    return stats

def drop_geometry(feature):
    return feature.setGeometry(None)

for year in years:
    print(f"Processing year {year}...")
    image = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate(f'{year}-01-01', f'{year+1}-01-01').first()
    if image is None:
        print(f"No FireCCI image for year {year}, skipping...")
        continue
    weeks = list(range(1, 54))  # 1 to 53
    # Map over all weeks for this year
    weekly_collections = ee.List(weeks).map(lambda w: features_for_week(image, year, ee.Number(w)))
    all_features = ee.FeatureCollection(weekly_collections).flatten()
    all_features_no_geom = all_features.map(drop_geometry)

    task = ee.batch.Export.table.toDrive(
        collection=all_features_no_geom.select(export_fields),
        description=f'FireCCI51_BurnedPixels_Catchments_Weekly_{year}',
        fileFormat='CSV'
    )
    task.start()
    print(f"Export started for {year} – Monitor in GEE Tasks or Google Drive.")

print("All exports started!")


Processing year 2002...
Export started for 2002 – Monitor in GEE Tasks or Google Drive.
Processing year 2003...
Export started for 2003 – Monitor in GEE Tasks or Google Drive.
Processing year 2004...
Export started for 2004 – Monitor in GEE Tasks or Google Drive.
Processing year 2005...
Export started for 2005 – Monitor in GEE Tasks or Google Drive.
Processing year 2006...
Export started for 2006 – Monitor in GEE Tasks or Google Drive.
Processing year 2007...
Export started for 2007 – Monitor in GEE Tasks or Google Drive.
Processing year 2008...
Export started for 2008 – Monitor in GEE Tasks or Google Drive.
Processing year 2009...
Export started for 2009 – Monitor in GEE Tasks or Google Drive.
Processing year 2010...
Export started for 2010 – Monitor in GEE Tasks or Google Drive.
Processing year 2011...
Export started for 2011 – Monitor in GEE Tasks or Google Drive.
Processing year 2012...
Export started for 2012 – Monitor in GEE Tasks or Google Drive.
Processing year 2013...
Export s

In [5]:
import pandas as pd
from datetime import datetime, timedelta
import os

def week_start_end(year, week):
    start = datetime(year, 1, 1) + timedelta(days=(week - 1) * 7)
    end = min(start + timedelta(days=6), datetime(year, 12, 31))
    return start.strftime('%Y/%m/%d'), end.strftime('%Y/%m/%d')

landcover_code_to_name = {
    0: "Cropland_rainfed",
    20: "Cropland_irrigated",
    30: "Mosaic_cropland",
    40: "Mosaic_natural_veg",
    50: "Tree_broad_evergreen",
    60: "Tree_broad_deciduous",
    70: "Tree_needle_evergreen",
    80: "Tree_needle_deciduous",
    90: "Tree_mixed",
    100: "Mosaic_tree_shrub",
    110: "Mosaic_herbaceous",
    120: "Shrubland",
    130: "Grassland",
    140: "Lichens_mosses",
    150: "Sparse_vegetation",
    170: "Tree_flooded_saline",
    180: "Shrub_herb_flooded"
}

folder = "Datasets/firecci_raw"  # Update if your CSVs are in a different folder

# Loop over all CSV files in the folder
for fname in os.listdir(folder):
    if fname.endswith('.csv') and fname.startswith('FireCCI51_BurnedPixels_Catchments_Weekly'):
        fpath = os.path.join(folder, fname)
        df = pd.read_csv(fpath)

        # Drop unwanted columns if present
        to_drop = ['system:index', '.geo']
        df = df.drop(columns=[c for c in to_drop if c in df.columns])

        # Add week_start and week_end columns
        df[['week_start', 'week_end']] = df.apply(
            lambda row: week_start_end(int(row['year']), int(row['week'])), axis=1, result_type='expand'
        )

        # Rename land cover columns to descriptive names
        rename_dict = {
            f"burned_pix_lc_{code}": f"{name}"
            for code, name in landcover_code_to_name.items()
        }
        df = df.rename(columns=rename_dict)

        # Reorder columns
        main_cols = ['Lake_ID', 'year', 'week', 'week_start', 'week_end']
        other_cols = [col for col in df.columns if col not in main_cols]
        df = df[main_cols + other_cols]

        # Sort by Lake_ID, year, week
        df = df.sort_values(by=['Lake_ID', 'year', 'week'])

        # Save (overwrite or save as new)
        df.to_csv(fpath, index=False)
        print(f"✅ Processed {fname}")

print("🎉 All weekly FireCCI tables updated with descriptive names and dates!")


✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2002.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2003.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2004.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2005.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2006.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2007.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2008.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2009.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2010.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2011.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2012.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2013.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2014.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2015.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2016.csv
✅ Processed FireCCI51_BurnedPixels_Catchments_Weekly_2017.csv
✅ Proces

In [1]:
import pandas as pd
import os

folder = "Datasets/firecci_raw"  # update if needed

# Find all FireCCI CSVs in the folder
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith('FireCCI51_BurnedPixels_Catchments_Weekly') and f.endswith('.csv')]

# Concatenate all years
dfs = [pd.read_csv(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

# Identify land cover columns
meta_cols = ['Lake_ID', 'year', 'week', 'week_start', 'week_end']
landcover_cols = [col for col in df_all.columns if col not in meta_cols]

# Aggregate burned pixels by land cover
total_burned_by_lc = df_all[landcover_cols].sum().sort_values(ascending=False)

# Optionally map codes to names
landcover_code_to_name = {
    0: "Cropland_rainfed", 20: "Cropland_irrigated", 30: "Mosaic_cropland",
    40: "Mosaic_natural_veg", 50: "Tree_broad_evergreen", 60: "Tree_broad_deciduous",
    70: "Tree_needle_evergreen", 80: "Tree_needle_deciduous", 90: "Tree_mixed",
    100: "Mosaic_tree_shrub", 110: "Mosaic_herbaceous", 120: "Shrubland",
    130: "Grassland", 140: "Lichens_mosses", 150: "Sparse_vegetation",
    170: "Tree_flooded_saline", 180: "Shrub_herb_flooded"
}
def get_name(col):
    for code, name in landcover_code_to_name.items():
        if str(code) in col:
            return name
    return col

summary_df = total_burned_by_lc.reset_index()
summary_df.columns = ['LandCover', 'TotalBurnedPixels']
summary_df['LandCoverName'] = summary_df['LandCover'].apply(get_name)

print(summary_df[['LandCover', 'LandCoverName', 'TotalBurnedPixels']])

print("\nTop 5 most affected land covers (all years):")
print(summary_df[['LandCoverName', 'TotalBurnedPixels']].head(5))


                LandCover          LandCoverName  TotalBurnedPixels
0               Grassland              Grassland       28249.882353
1    Tree_broad_deciduous   Tree_broad_deciduous       23558.815686
2      Shrub_herb_flooded     Shrub_herb_flooded       17873.615686
3               Shrubland              Shrubland       16407.917647
4   Tree_needle_evergreen  Tree_needle_evergreen       13591.498039
5      Mosaic_natural_veg     Mosaic_natural_veg        8187.513725
6              Tree_mixed             Tree_mixed        4983.282353
7       Mosaic_herbaceous      Mosaic_herbaceous        3896.486275
8         Mosaic_cropland        Mosaic_cropland        3877.368627
9    Tree_broad_evergreen   Tree_broad_evergreen        2554.760784
10      Mosaic_tree_shrub      Mosaic_tree_shrub        1870.474510
11  Tree_needle_deciduous  Tree_needle_deciduous        1557.098039
12    Tree_flooded_saline    Tree_flooded_saline         512.941176
13      Sparse_vegetation      Sparse_vegetation